<a href="https://colab.research.google.com/github/boradj/machine-learning/blob/main/food-vision/food_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Important stuff

In [1]:
##Mixed precision

## Check GPU

In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-06ebfa52-ebdf-b37a-bac2-b84d87afe408)


##helper function

In [3]:
# download it

In [4]:
## use tensorflow datasets
import tensorflow_datasets as tfds


In [5]:
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [6]:
(train_data, test_data), ds_info = tfds.load(name = "food101",
                                    split=["train", "validation"],
                                    shuffle_files=True,
                                    as_supervised = True,
                                    with_info= True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/food101/2.0.0.incompleteT9FSNB/food101-train.tfrecord*...:   0%|          | 0/…

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/food101/2.0.0.incompleteT9FSNB/food101-validation.tfrecord*...:   0%|         …

Dataset food101 downloaded and prepared to ~/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [7]:
# Features of 101
ds_info.features


FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [8]:
class_names = ds_info.features["label"].names


In [9]:
train_data
train_one_sample = train_data.take(1)

In [10]:
train_one_sample

<TakeDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [11]:
for image, label in train_one_sample:
  print(f"""
        Image shape:{image.shape},
        Image datatype:{image.dtype},
        Target class from Food101(tensor form):{label}
        class_name (str form):{class_name[label.numpy()]}""")

NameError: ignored

In [ ]:
image

In [ ]:
import tensorflow as tf
tf.reduce_min(image), tf.reduce_max(image)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False);